In [1]:
from calliope.data import DataManager
from calliope.market import MeritOrderNEM, DataToBidObjInterface
import pandas as pd
import sqlite3
con = sqlite3.connect("E:\Code\Calliope/notebooks\historical_mms.db")
import datetime
data_manager = DataManager(con)
import matplotlib.pyplot as plt
import pickle

In [2]:
from calliope.data import get_fcas_requirements_region_assumed
start_date = '2019/01/01 00:00:00'
end_date = '2019/04/01 00:00:00'
fcas_req = get_fcas_requirements_region_assumed(con, start_date, end_date, region='NSW1')

In [10]:
# shit code i hate this
import pickle
from calliope.data import DataManager
from calliope.market import DataToBidObjInterface
import pandas as pd
import sqlite3
con = sqlite3.connect("C:/Users/YOUR_PATH/historical_mms.db")
import datetime
data_manager = DataManager(con)

mnths = range(1,11)
months = ['jan', 'feb', 'mar','apr', 'may', 'jun', 'jul','aug','sep','oct']
for mnth, mn in zip(mnths, months):
    def tomnthstr(mnth):
        return f'0{mnth}' if mnth <10 else f'{mnth}'

    print(f'runnin month {tomnthstr(mnth)}')
    start_date = f'2019/{tomnthstr(mnth)}/01 00:00:00'
    end_date = f'2019/{tomnthstr(mnth+1)}/01 00:00:00'
    region = 'NSW1'
    print(f'loading data')

    data_manager.load_data(start_date, end_date, region)

    print(f'creating bids')
    data_to_bid = DataToBidObjInterface(data_manager)
    bid_dict = data_to_bid.data_to_bids()

    print(f'saving 2 file bids')

    with open(f'bid_dict_{mn}_nsw.pkl', 'wb') as f:
        pickle.dump(bid_dict, f, pickle.HIGHEST_PROTOCOL)
    print('done')


runnin month 04
loading data
creating bids
saving 2 file bids
done
runnin month 05
loading data
creating bids
saving 2 file bids
done
runnin month 06
loading data
creating bids
saving 2 file bids
done
runnin month 07
loading data
creating bids
saving 2 file bids
done
runnin month 08
loading data
creating bids
saving 2 file bids
done
runnin month 09
loading data
creating bids
saving 2 file bids
done
runnin month 10
loading data
creating bids
saving 2 file bids
done


In [11]:
# Create the merit order raw prices to train over
start_date = '2019/01/01 00:00:00'
end_date = '2019/11/01 00:00:00'
region = 'NSW1'

data_manager.load_data(start_date, end_date, region)
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul','aug','sep','oct']
bid_dict_files=[f'E:/Code/Calliope/notebooks/bid_dict_{i}_nsw.pkl' for i in months]
nem = MeritOrderNEM(data_manager)
nem.build()
nem.load_bids_from_pickle(bid_dict_files)

from calliope.market import MARKETS

prices = {m: [] for m in MARKETS}
prices['timestamp'] = []

# run thru and append market prices
i = 0
while not nem.terminated:
    i+=1
    if not i%288:
        print(f'Current: {nem.data_manager.dispatch_intervals[nem.timestep]}')
        print(f'DI {i}')
    nem.add_bids()
    dispatch = nem.dispatch()
    prices['timestamp'].append(nem.data_manager.dispatch_intervals[nem.timestep])
    nem.step()
    for m in MARKETS:
        prices[m].append(dispatch[m]['price'])
nem.reset()

# Save to csv
pd.DataFrame(prices).to_csv('merit_order_prices.csv', index=False)

In [15]:
# save outcome prices for later
with open('merit_order_prices.pkl', 'wb') as f:
    pickle.dump(prices, f, pickle.HIGHEST_PROTOCOL)

In [17]:
prices_df = pd.read_csv('merit_order_prices.csv')

In [42]:
nem.markets['ENERGY'].dispatch(verbose=True)

Evaluating UPPTUMUT, 1: <-1011.2,0.0>
Bid accepted. Dispatched for 0.0. Cumulative capacity now 0.0
Evaluating WOODLWN1, 1: <-1010.0,48.0>
Bid accepted. Dispatched for 48.0. Cumulative capacity now 48.0
Evaluating WOODLWN1, 2: <-1008.99,0.0>
Bid accepted. Dispatched for 0.0. Cumulative capacity now 48.0
Evaluating STWF1, 1: <-1006.2,198.0>
Bid accepted. Dispatched for 198.0. Cumulative capacity now 246.0
Evaluating COLEASF1, 1: <-1001.9,150.0>
Bid accepted. Dispatched for 150.0. Cumulative capacity now 396.0
Evaluating DG_NSW1, 1: <-1000.0,0.0>
Bid accepted. Dispatched for 0.0. Cumulative capacity now 396.0
Evaluating TALWA1, 1: <-999.5,190.0>
Bid accepted. Dispatched for 190.0. Cumulative capacity now 586.0
Evaluating SHGEN, 1: <-998.1,0.0>
Bid accepted. Dispatched for 0.0. Cumulative capacity now 586.0
Evaluating SITHE01, 1: <-997.51,0.0>
Bid accepted. Dispatched for 0.0. Cumulative capacity now 586.0
Evaluating PARSF1, 1: <-996.4,50.0>
Bid accepted. Dispatched for 50.0. Cumulative c

{'dispatch': {'UPPTUMUT': 0.0,
  'WOODLWN1': 48.0,
  'STWF1': 198.0,
  'COLEASF1': 150.0,
  'DG_NSW1': 0.0,
  'TALWA1': 190.0,
  'SHGEN': 0.0,
  'SITHE01': 0.0,
  'PARSF1': 50.0,
  'TUMUT3': 0.0,
  'BLOWERNG': 80.0,
  'TARALGA1': 106.0,
  'CROOKWF2': 88.0,
  'ER03': 210.0,
  'ER04': 210.0,
  'VP5': 280.0,
  'VP6': 250.0,
  'ER01': 210.0,
  'ER02': 210.0,
  'CG1': 0.0,
  'CG2': 0.0,
  'CG3': 0.0,
  'CG4': 0.0,
  'BODWF1': 110.0,
  'BROKENH1': 53.0,
  'BOCORWF1': 113.0,
  'MP1': 350.0,
  'MP2': 350.0,
  'GUNNING1': 47.0,
  'URANQ11': 0.0,
  'URANQ12': 0.0,
  'URANQ13': 0.0,
  'URANQ14': 0.0,
  'HVGTS': 0.0,
  'NYNGAN1': 102.0,
  'BW03': 250.0,
  'BW04': 250.0,
  'BW01': 350.0,
  'BW02': 250.0,
  'GUTHEGA': 0.0,
  'LD01': 350.0,
  'LD02': 250.0,
  'LD03': 220.0,
  'LD04': 250.0,
  'HUMENSW': 70.0,
  'MOREESF1': 56.0,
  'SAPHWF1': 270.0,
  'GULLRWF1': 0.0,
  'GULLRSF1': 10.0,
  'MANSLR1': 46.0,
  'WRSF1': 20.0,
  'WRWF1': 155.09000000000015},
 'price': 0.0,
 'demand': 6202.09,
 'all_bids':

In [28]:
dispatch['ENERGY']['dispatch']

{'UPPTUMUT': 0.0,
 'WOODLWN1': 48.0,
 'STWF1': 198.0,
 'COLEASF1': 150.0,
 'DG_NSW1': 0.0,
 'TALWA1': 190.0,
 'SHGEN': 0.0,
 'SITHE01': 0.0,
 'PARSF1': 50.0,
 'TUMUT3': 0.0,
 'BLOWERNG': 80.0,
 'TARALGA1': 106.0,
 'CROOKWF2': 88.0,
 'ER03': 210.0,
 'ER04': 210.0,
 'VP5': 280.0,
 'VP6': 250.0,
 'ER01': 210.0,
 'ER02': 210.0,
 'CG1': 0.0,
 'CG2': 0.0,
 'CG3': 0.0,
 'CG4': 0.0,
 'BODWF1': 110.0,
 'BROKENH1': 53.0,
 'BOCORWF1': 113.0,
 'MP1': 350.0,
 'MP2': 350.0,
 'GUNNING1': 47.0,
 'URANQ11': 0.0,
 'URANQ12': 0.0,
 'URANQ13': 0.0,
 'URANQ14': 0.0,
 'HVGTS': 0.0,
 'NYNGAN1': 102.0,
 'BW03': 250.0,
 'BW04': 250.0,
 'BW01': 350.0,
 'BW02': 250.0,
 'GUTHEGA': 0.0,
 'LD01': 350.0,
 'LD02': 250.0,
 'LD03': 220.0,
 'LD04': 250.0,
 'HUMENSW': 70.0,
 'MOREESF1': 56.0,
 'SAPHWF1': 270.0,
 'GULLRWF1': 0.0,
 'GULLRSF1': 10.0,
 'MANSLR1': 46.0,
 'WRSF1': 20.0,
 'WRWF1': 115.55000000000018}

In [ ]:
if False:
    import pickle
    with open('data_manager.pkl', 'wb') as f:
        pickle.dump(data_manager, f, pickle.HIGHEST_PROTOCOL)
    with open('data_manager.pkl', 'rb') as f:
        test_data_manager = pickle.load(f)

In [ ]:
bid_dict = {d: {} for d in data_manager.dispatch_intervals}

In [ ]:
curr_di = data_manager.dispatch_intervals[0]

In [ ]:
demand_dict

{'RAISEREG': 39.525,
 'LOWERREG': 50.0,
 'RAISE6SEC': 183.6,
 'RAISE60SEC': 204.6,
 'RAISE5MIN': 200.0,
 'LOWER6SEC': 54.0,
 'LOWER60SEC': 76.30478,
 'LOWER5MIN': 74.0,
 'ENERGY': 6162.55}

In [ ]:
data_manager.regional_demand.loc[data_manager.regional_demand.SETTLEMENTDATE == data_manager.dispatch_intervals[0]]

,SETTLEMENTDATE,REGIONID,TOTALDEMAND,DEMANDFORECAST,INITIALSUPPLY
49,2019-01-01 04:05:00,NSW1,6162.55,-10.00781,6794.14111


In [ ]:
volume_bids = data_manager.volume_bids
price_bids = data_manager.price_bids



# Interval datetime is the end of the dispatch interval. 
# this means midnight actually belongs to the previous day, so need to account when creating settlementdate
volume_bids['SETTLEMENTDATE'] = pd.to_datetime(
    volume_bids.INTERVAL_DATETIME.dt.date - datetime.timedelta(seconds=1)
)

pq_bids = pd.merge(volume_bids, price_bids, how='left', on=['SETTLEMENTDATE', 'DUID', 'BIDTYPE'])


In [ ]:
pq_bids

,INTERVAL_DATETIME,DUID,BIDTYPE,BANDAVAIL1,BANDAVAIL2,BANDAVAIL3,BANDAVAIL4,BANDAVAIL5,BANDAVAIL6,BANDAVAIL7,...,PRICEBAND1,PRICEBAND2,PRICEBAND3,PRICEBAND4,PRICEBAND5,PRICEBAND6,PRICEBAND7,PRICEBAND8,PRICEBAND9,PRICEBAND10
0,2019-01-01 04:05:00,ASNACTW1,LOWER5MIN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.0
1,2019-01-01 04:05:00,ASNACTW1,LOWER60SEC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.0
2,2019-01-01 04:05:00,ASNACTW1,LOWER6SEC,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.0
3,2019-01-01 04:05:00,ASNENC1,RAISE5MIN,18.0,0.0,18.0,18.0,14.0,4.0,0.0,...,0.03,0.30,0.49,0.99,1.98,5.00,9.90,17.80,100.00,10000.0
4,2019-01-01 04:05:00,ASNENC1,RAISE60SEC,12.0,0.0,13.0,12.0,10.0,2.0,0.0,...,0.03,0.30,0.73,0.99,1.98,5.00,9.90,18.05,100.00,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1953815,2019-02-01 00:05:00,VP6,RAISE6SEC,10.0,10.0,5.0,5.0,3.0,0.0,0.0,...,1.00,2.00,5.00,10.00,15.00,30.00,50.00,300.00,4885.00,13800.0
1953816,2019-02-01 00:05:00,VP6,RAISEREG,0.0,0.0,0.0,5.0,0.0,5.0,0.0,...,1.00,2.00,7.00,14.00,28.00,30.00,48.00,100.00,300.00,13800.0
1953817,2019-02-01 00:05:00,WOODLWN1,ENERGY,48.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1010.00,-1008.99,-115.14,-80.80,-60.60,-2.02,299.97,5036.87,12935.21,14645.0
1953818,2019-02-01 00:05:00,WRSF1,ENERGY,20.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,1.00,2.00,4.00,8.00,16.00,32.00,64.00,128.00,256.0


In [ ]:
# Melt the BANDAVAIL columns
PRICE_BID_COLS = [
    'PRICEBAND1', 'PRICEBAND2', 'PRICEBAND3', 'PRICEBAND4', 'PRICEBAND5', 
    'PRICEBAND6', 'PRICEBAND7', 'PRICEBAND8', 'PRICEBAND9', 'PRICEBAND10'
]

VOLUME_BID_COLS = [
    'BANDAVAIL1', 'BANDAVAIL2', 'BANDAVAIL3', 'BANDAVAIL4', 'BANDAVAIL5',
    'BANDAVAIL6', 'BANDAVAIL7', 'BANDAVAIL8', 'BANDAVAIL9', 'BANDAVAIL10'
 ]

# Melt the PRICEBAND columns
melted_priceband = pq_bids.melt(id_vars=['INTERVAL_DATETIME', 'DUID', 'BIDTYPE', 'DISPATCHTYPE'], 
                        value_vars=PRICE_BID_COLS, 
                        var_name='PRICEBAND', 
                        value_name='PRICE_BID')

# Melt the BANDAVAIL columns
melted_bandavail = pq_bids.melt(id_vars=['INTERVAL_DATETIME', 'DUID', 'BIDTYPE', 'DISPATCHTYPE'], 
                        value_vars=VOLUME_BID_COLS, 
                        var_name='BANDAVAIL', 
                        value_name='QUANTITY_BID')

# Get just the band number itself
melted_priceband['BAND'] = melted_priceband['PRICEBAND'].str.extract('(\d+)').astype(int)
melted_bandavail['BAND'] = melted_bandavail['BANDAVAIL'].str.extract('(\d+)').astype(int)

# Merge the melted dataframes
merged_df = pd.merge(melted_bandavail, melted_priceband, 
                    left_on=['INTERVAL_DATETIME', 'DUID', 'BIDTYPE', 'DISPATCHTYPE', 'BAND'],
                    right_on=['INTERVAL_DATETIME', 'DUID', 'BIDTYPE', 'DISPATCHTYPE', 'BAND'])

final_columns = ['INTERVAL_DATETIME', 'DUID', 'BIDTYPE', 'DISPATCHTYPE', 'BAND', 'PRICE_BID', 'QUANTITY_BID']
final_df = merged_df[final_columns]

pq_bids_melted = final_df

In [ ]:
bid_dict = {d: {} for d in pq_bids_melted.INTERVAL_DATETIME.unique().tolist()}

In [ ]:
from calliope.market import Bid

In [ ]:
MARKETS = ['ENERGY', 'RAISEREG', "LOWERREG", "RAISE6SEC", "RAISE60SEC", "RAISE5MIN", "LOWER6SEC", "LOWER60SEC", "LOWER5MIN"]

i=0
for key_datetime in bid_dict:

    pq_bid_i = pq_bids_melted.loc[pq_bids_melted.INTERVAL_DATETIME == key_datetime]

    # Obscene list comprehension
    # Create a dictionary indexed by the market, where the values are lists of Bid objects.
    bid_dict_i = {
        m: pq_bid_i.loc[pq_bid_i.BIDTYPE == m]
            .apply(
                lambda x: Bid(x.DUID, x.PRICE_BID, x.QUANTITY_BID, x.BAND), 
                axis=1
            ).tolist() 
            for m in MARKETS
        }

    bid_dict[key_datetime] = bid_dict_i
    i+=1

    if i > 100:
        break


In [ ]:
import pickle
with open('bid_dict.pkl', 'wb') as f:
    pickle.dump(bid_dict, f, pickle.HIGHEST_PROTOCOL)